<a href="https://colab.research.google.com/github/lesley880813/specialhw2/blob/master/cifar10_groupconv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Model
from keras.layers import Input 
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.merge import concatenate
from keras.layers import Dropout
from keras.datasets import cifar10
from keras.layers import MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
   
    # one hot encode target values
    print(trainX.shape)
    print(trainY.shape)
    '''
    for i in range(50000):
        for x in range(0,16):
            for y in range(0,16):
                for z in range(0,3):
                    trainX[i][x][y][z]=trainX[i][2*x][2*y][z]-(trainX[i][2*x][2*y][z]%32)
                    if i<10000 :
                      testX[i][x][y][z]=testX[i][2*x][2*y][z]-(testX[i][2*x][2*y][z]%32)
    
    '''                
   
        
    
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    #return trainX[:,0:16,0:16,:], trainY, testX[:,0:16,0:16,:], testY
    return trainX, trainY, testX, testY



def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm/255.0
    test_norm = test_norm/255.0
    # return normalized images
    return train_norm, test_norm



#model definition
def define_model():
  input = Input(shape=(32,32,3))

  conv11 = Conv2D(16, (3, 3),strides=3, padding='same',input_shape=(32, 32, 1),data_format="channels_last",activation='relu')(input[:,:,:,1:2])
  drop1 = Dropout(0.1)(conv11)

  max1 = MaxPooling2D(pool_size=(2,2),strides=3, padding='same')(input[:,:,:,0:1])
  max2 = MaxPooling2D(pool_size=(2,2),strides=3, padding='same')(input[:,:,:,2:3])



  merge = concatenate([drop1,max1,max2],axis=-1)

  conv21 = Conv2D(21, (2, 2),strides=2,activation='relu')(merge[:,:,:,0:9])
  conv22 = Conv2D(21, (2, 2),strides=2,activation='relu')(merge[:,:,:,6:15])
  conv23 = Conv2D(16, (2, 2),strides=2,activation='relu')(merge[:,:,:,12:18])



  drop21 = Dropout(0.1)(conv21)
  drop22 = Dropout(0.1)(conv22)
  drop23 = Dropout(0.1)(conv23)

  flat1 = Flatten()(drop21)

  flat2 = Flatten()(drop22)

  flat3 = Flatten()(drop23)

  merge_last= concatenate([flat1,flat2,flat3])


  output = Dense(10,activation='softmax')(merge_last)

  model = Model(inputs=input,outputs=output)
  model.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=['accuracy'])


  return model


def train_with_original_data(model):

  trainX, trainY, testX, testY = load_dataset()
  #trainX, testX = prep_pixels(trainX, testX)
  model.fit(trainX, trainY , batch_size=200, epochs=100, validation_split=0.1)
  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))

def train_with_augmented_data(model):

  trainX, trainY, testX, testY = load_dataset()
  datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1)
  
  datagen.fit(trainX)
  model.fit(datagen.flow(trainX, trainY, batch_size=200,
         subset='training'), 
          validation_data=datagen.flow(trainX, trainY,
         batch_size=15, subset='validation'), epochs=100)

  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))

def view_weights(model):
  for layer in model.layers: print(layer.get_weights())


def save_model(model):
  model.save("/content/gdrive/MyDrive/MyModel_tf",save_format='tf')




#model = define_model()
#model.summary()
train_with_original_data(model)

#train_with_augmented_data(model)

'''
dic = {0: 'airplane', 1: 'automobile', 2: 'bird', 3:'cat' ,4:'deer',5:'dog',6:'frog',7:'horse',8:'ship',9:'truck'}
total=[0,0,0,0,0,0,0,0,0,0]
correct=[0,0,0,0,0,0,0,0,0,0]


for i in range(0,1000,2):
  # Generate a plot
  #reshaped_image = testX[sample].reshape((img_width, img_height))
  if i<10 :
    plt.imshow(testX[i])
    print(testY[i])
    plt.imshow(testX[i+1])
    print(testY[i+1])
  
  
  data = np.stack((testX[i],testX[i+1]))

  predictions = model.predict(data)
  if i<10 :
    print(predictions)

  classes = np.argmax(predictions, axis = 1)
  total[classes[0]]+=1
  total[classes[1]]+=1

  if testY[i][classes[0]]==1:
    correct[classes[0]]+=1
  if testY[i+1][classes[0]]==1:
    correct[classes[1]]+=1



  #if 
  # Add sample to array for prediction
  

print(total)
print(correct)
'''





(50000, 32, 32, 3)
(50000, 1)
Epoch 1/100
225/225 [==============================] - 11s 48ms/step - loss: 1.0022 - accuracy: 0.6460 - val_loss: 1.0638 - val_accuracy: 0.6442
Epoch 2/100
225/225 [==============================] - 10s 46ms/step - loss: 1.0022 - accuracy: 0.6469 - val_loss: 1.0953 - val_accuracy: 0.6356
Epoch 3/100
225/225 [==============================] - 10s 46ms/step - loss: 0.9984 - accuracy: 0.6474 - val_loss: 1.0908 - val_accuracy: 0.6326
Epoch 4/100
225/225 [==============================] - 10s 47ms/step - loss: 0.9880 - accuracy: 0.6540 - val_loss: 1.0748 - val_accuracy: 0.6438
Epoch 5/100
225/225 [==============================] - 11s 47ms/step - loss: 0.9924 - accuracy: 0.6497 - val_loss: 1.0778 - val_accuracy: 0.6408
Epoch 6/100
225/225 [==============================] - 10s 47ms/step - loss: 0.9922 - accuracy: 0.6494 - val_loss: 1.1042 - val_accuracy: 0.6308
Epoch 7/100
225/225 [==============================] - 10s 47ms/step - loss: 1.0023 - accuracy: 0.64

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
#drive.mount('/content/gdrive')

In [9]:
# Round off weights to the second decimal space
def round_off_weight(model):
  for layer in model.layers: 

    temp = layer.get_weights()
    if len(temp)==0:
      continue

    rounded =[]
    for count in range(0,len(temp)):
      
      
      a = np.around(temp[count].astype(np.double) , decimals=2)
      rounded.append(a)

    layer.set_weights(rounded)
    


  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))


In [ ]:
  #just evaluate
  trainX, trainY, testX, testY = load_dataset()
  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))

(50000, 32, 32, 3)
(50000, 1)
> 62.110


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 32, 32, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 11, 11, 16)   160         tf.__operators__.getitem[0][0]   
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 32, 32, 1)    0           input_1[0][0]                    
______________________________________________________________________________________________